
# WWLayerIterator

This Notebook explains how to use the internal WeightWatcher Layer Iterators







In [1]:
# Suppress the powerlaw package warnings
# "powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide"
# "powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide"
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext watermark
%watermark

2021-07-31T15:29:26-07:00

CPython 3.7.4
IPython 7.17.0

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 17.7.0
machine    : x86_64
processor  : i386
CPU cores  : 12
interpreter: 64bit


### Import WeightWatcher

set custom Logging at WARN Level

In [3]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARN)

import weightwatcher as ww
import torchvision.models as models


ww.__version__

'0.4.9'

###  How to create a WWLayerIterator

In [4]:
model = models.vgg19_bn(pretrained=True)
watcher = ww.WeightWatcher(model=model)
ww_layer_iterator = watcher.make_layer_iterator(model=model)

INFO:weightwatcher:

python      version 3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
numpy       version 1.19.1
tensforflow version 2.1.0
keras       version 2.2.4-tf
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': True, 'savefig': False, 'rescale': True, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False}


### The Iterator lets you loop over WWLayer instances

- The WWLayer instance (object) is a wrapper to the underlying framework layers

- The intent is to only access the WWLayer instance and not the underlying framework methods

- This lets weightwatcher apply different functions / transformations on each layer individually



In [5]:
for ww_layer in ww_layer_iterator:
    print(ww_layer)

WWLayer 2  None PYTORCH CONV2D  skipped False
WWLayer 5  None PYTORCH CONV2D  skipped False
WWLayer 9  None PYTORCH CONV2D  skipped False
WWLayer 12  None PYTORCH CONV2D  skipped False
WWLayer 16  None PYTORCH CONV2D  skipped False
WWLayer 19  None PYTORCH CONV2D  skipped False
WWLayer 22  None PYTORCH CONV2D  skipped False
WWLayer 25  None PYTORCH CONV2D  skipped False
WWLayer 29  None PYTORCH CONV2D  skipped False
WWLayer 32  None PYTORCH CONV2D  skipped False
WWLayer 35  None PYTORCH CONV2D  skipped False
WWLayer 38  None PYTORCH CONV2D  skipped False


WWLayer 42  None PYTORCH CONV2D  skipped False
WWLayer 45  None PYTORCH CONV2D  skipped False
WWLayer 48  None PYTORCH CONV2D  skipped False
WWLayer 51  None PYTORCH CONV2D  skipped False
WWLayer 57  None PYTORCH DENSE  skipped False
WWLayer 60  None PYTORCH DENSE  skipped False
WWLayer 63  None PYTORCH DENSE  skipped False


### The .__dict__ method lets you inspect what is actually in the iterator

In [6]:
li.__dict__

{'params': {'glorot_fix': False,
  'normalize': False,
  'conv2d_norm': True,
  'randomize': True,
  'savefig': False,
  'rescale': True,
  'deltaEs': False,
  'intra': False,
  'channels': None,
  'conv2d_fft': False,
  'ww2x': False},
 'k': 0,
 'model': VGG(
   (features): Sequential(
     (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): ReLU(inplace=True)
     (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (5): ReLU(inplace=True)
     (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (9): ReLU(inplace=True)
     (10): Conv2d(128, 128, kernel_size=(

### The WWLayerIterator constructor method takes

- layers=[LAYER_ID, ...] to specify filters, as in the watcher.analyze(..,) and watcher.describe(...) methods

- other parameters, like ww2x and channels, are specified in the parameters dict



In [18]:
logger = logging.getLogger('weightwatcher') 
logger.setLevel(logging.DEBUG)


layers = [60]
DEFAULT_PARAMS = {'glorot_fix': False, 'normalize':False, 'conv2d_norm':True, 'randomize': True, 'savefig':False, 
                  'rescale':True , 'deltaEs':False, 'intra':False, 'channels':None, 'conv2d_fft':False, 
                  'ww2x':False}

params = DEFAULT_PARAMS
ww_layer_iterator = watcher.make_layer_iterator(model=model, layers=layers, params=params)

INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': True, 'savefig': False, 'rescale': True, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False}
DEBUG:weightwatcher:FRAMEWORKS: KERAS = 4  PYTORCH = 2 ONNX = 8 UNKNOWN = 1 
DEBUG:weightwatcher:FIRST = 2  LAST = 4 UNKNOWN = 1 
DEBUG:weightwatcher:MODEL ITERATOR, framework = 2, channels = 4 
INFO:weightwatcher:Filtering layer by id 60


### WWLayer  Instances

When a WWLayer instance is created, the weight matrices for the layer are extracted from the underlying framework tensor (i.e. layer.weights and layer.biases) and placed into WMats

- WMats = [W,W,W,...] contains 1 or more W matrices, of the same shape NxM, N > M
- evals: the *combined* evals for each layer
- rf = 1 or (k)x(k)  the size of the 'receptive field'



- layer_type =  an internal enum:    known layer types so far include:

<pre>
LAYER_TYPE.DENSE | LAYER_TYPE.CONV1D | LAYER_TYPE.CONV2D | LAYER_TYPE.FLATTENED |  LAYER_TYPE.EMBEDDING | LAYER_TYPE.NORM
</pre>

- channel_str:  string for channel type :  "FIRST" | "LAST" | "UNKNOWN"



### WeightWatcher Apply Methods

#### The various apply_xxx() methods use basic metaprogramming to set additional instance variables


- apply_filters()
- apply_normalize_Wmats()
- apply_esd()
- apply_random_esd()
- apply_plot_esd()
- apply_fit_powerlaw()
- apply_norm_metrics()
- apply_plot_deltaEs()
- apply_mp_fit()
- apply_svdsmoothing()


i.e the apply_esd() method runs SVD on all the WMats, then combines them into a single ESD

<code>
#
    def apply_esd(self, ww_layer, params=AULT_PARAMS):
            """run full SVD on layer weight matrices, compute ESD on combined eigenvalues, combine all..."""
#
...
#
    ww_layer.evals = evals
    ww_layer.add_column("has_esd", True)
    ww_layer.add_column("num_evals", len(evals))
    ww_layer.add_column("sv_max", sv_max)
    ww_layer.add_column("rank_loss", rank_loss)
    ww_layer.add_column("lambda_max", np.max(evals))    
#      
    return ww_layer
</code>       
        
